### JSON Parsing And Processing

In [1]:
import json
import os
os.makedirs('data/json_files', exist_ok=True)

In [2]:
json_data = {
    "company": "TechCorp",
    "employees": [
        {
            "id": 1,
            "name": "John Doe",
            "role": "Software Engineer",
            "skills": ["Python", "JavaScript", "React"],
            "projects": [
                {"name": "RAG System", "status": "In Progress"},
                {"name": "Data Pipeline", "status": "Completed"}
            ]
        },
        {
            "id": 2,
            "name": "Jane Smith",
            "role": "DevOps Engineer",
            "skills": ["Docker", "Kubernetes", "AWS"],
            "projects": [
                {"name": "Cloud Migration", "status": "In Progress"},
                {"name": "CI/CD Setup", "status": "Completed"}
            ]
        },
        {
            "id": 3,
            "name": "Mike Johnson",
            "role": "Data Scientist",
            "skills": ["Python", "PyTorch", "SQL"],
            "projects": [
                {"name": "Customer Churn Model", "status": "In Review"},
                {"name": "Sales Forecasting", "status": "Planning"}
            ]
        },
        {
            "id": 4,
            "name": "Sarah Williams",
            "role": "Product Manager",
            "skills": ["Agile", "Jira", "Roadmapping"],
            "projects": [
                {"name": "Q4 Roadmap", "status": "Completed"},
                {"name": "User Research", "status": "In Progress"}
            ]
        },
        {
            "id": 5,
            "name": "David Brown",
            "role": "Frontend Developer",
            "skills": ["React", "TypeScript", "Tailwind"],
            "projects": [
                {"name": "Dashboard Redesign", "status": "In Progress"}
            ]
        }
    ]
}

In [3]:
json_data

{'company': 'TechCorp',
 'employees': [{'id': 1,
   'name': 'John Doe',
   'role': 'Software Engineer',
   'skills': ['Python', 'JavaScript', 'React'],
   'projects': [{'name': 'RAG System', 'status': 'In Progress'},
    {'name': 'Data Pipeline', 'status': 'Completed'}]},
  {'id': 2,
   'name': 'Jane Smith',
   'role': 'DevOps Engineer',
   'skills': ['Docker', 'Kubernetes', 'AWS'],
   'projects': [{'name': 'Cloud Migration', 'status': 'In Progress'},
    {'name': 'CI/CD Setup', 'status': 'Completed'}]},
  {'id': 3,
   'name': 'Mike Johnson',
   'role': 'Data Scientist',
   'skills': ['Python', 'PyTorch', 'SQL'],
   'projects': [{'name': 'Customer Churn Model', 'status': 'In Review'},
    {'name': 'Sales Forecasting', 'status': 'Planning'}]},
  {'id': 4,
   'name': 'Sarah Williams',
   'role': 'Product Manager',
   'skills': ['Agile', 'Jira', 'Roadmapping'],
   'projects': [{'name': 'Q4 Roadmap', 'status': 'Completed'},
    {'name': 'User Research', 'status': 'In Progress'}]},
  {'id':

In [4]:
with open('data/json_files/company_data.json', 'w') as f:
    json.dump(json_data, f, indent=2)

In [5]:
# Save JSON Lines format
jsonl_data = [
    {"timestamp": "2024-01-01", "event": "user_login", "user_id": 123},
    {"timestamp": "2024-01-01", "event": "page_view", "user_id": 123, "page": "/home"},
    {"timestamp": "2024-01-01", "event": "purchase", "user_id": 123, "amount": 99.99}
]

with open('data/json_files/events.jsonl', 'w') as f:
    for item in jsonl_data:
        f.write(json.dumps(item) + '\n')

### JSON Processing Strategies

In [7]:
from langchain_community.document_loaders import JSONLoader
import json

## MEthod1 : JsonLoader With jq_schema
print("[1] JSONLoader - Extract specific fields")

# Extract employee information
employee_loader = JSONLoader(
    file_path='data/json_files/company_data.json',
    jq_schema='.employees[]',  # jq query to extract each employee
    text_content=False  # Get full JSON objects
)

employee_docs = employee_loader.load()
print(f"Loaded {len(employee_docs)} employee documents")
print(f"First employee: {employee_docs[0].page_content[:200]}...")
print(employee_docs)

[1] JSONLoader - Extract specific fields
Loaded 5 employee documents
First employee: {"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status"...
[Document(metadata={'source': 'C:\\Users\\aelouard\\OneDrive - NTT DATA EMEAL\\Desktop\\RAGUdemy\\0-DatalngestParsing\\data\\json_files\\company_data.json', 'seq_num': 1}, page_content='{"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status": "Completed"}]}'), Document(metadata={'source': 'C:\\Users\\aelouard\\OneDrive - NTT DATA EMEAL\\Desktop\\RAGUdemy\\0-DatalngestParsing\\data\\json_files\\company_data.json', 'seq_num': 2}, page_content='{"id": 2, "name": "Jane Smith", "role": "DevOps Engineer", "skills": ["Docker", "Kubernetes", "AWS"], "projects": [{"nam

In [8]:
# Method 2: Custom JSON processing for complex structures
from typing import List
from langchain_core.documents import Document
print("\n[2] Custom JSON Processing")

def process_json_intelligently(filepath: str) -> List[Document]:
    """Process JSON with intelligent flattening and context preservation"""
    with open(filepath, 'r') as f:
        data = json.load(f)

    documents = []

    # Strategy 1: Create documents for each employee with full context
    for emp in data.get('employees', []):
        content = f"""Employee Profile:
        Name: {emp['name']}
        Role: {emp['role']}
        Skills: {', '.join(emp['skills'])}

        Projects:"""
        for proj in emp.get('projects', []):
            content += f"\n- {proj['name']} (Status: {proj['status']})"

        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'data_type': 'employee_profile',
                'employee_id': emp['id'],
                'employee_name': emp['name'],
                'role': emp['role']
            }
        )
        documents.append(doc)
    
    # Note: The function likely returns 'documents' here, though it's cut off in the image.
    return documents


[2] Custom JSON Processing


In [9]:
process_json_intelligently('data/json_files/company_data.json')

[Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 1, 'employee_name': 'John Doe', 'role': 'Software Engineer'}, page_content='Employee Profile:\n        Name: John Doe\n        Role: Software Engineer\n        Skills: Python, JavaScript, React\n\n        Projects:\n- RAG System (Status: In Progress)\n- Data Pipeline (Status: Completed)'),
 Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 2, 'employee_name': 'Jane Smith', 'role': 'DevOps Engineer'}, page_content='Employee Profile:\n        Name: Jane Smith\n        Role: DevOps Engineer\n        Skills: Docker, Kubernetes, AWS\n\n        Projects:\n- Cloud Migration (Status: In Progress)\n- CI/CD Setup (Status: Completed)'),
 Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 3, 'employee_name': 'Mike Johnson', 'role': 'Data Scientist'}, page_cont